<a href="https://colab.research.google.com/github/Mehreen1103/CUET_ChiSquare/blob/main/Copy_of_FakeNewsDetection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

train=pd.read_csv("/content/Fake_train .csv")
train2=pd.read_csv("/content/Fake_dev.csv")

In [ ]:
data = train

In [ ]:
data

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original
...,...,...
3252,വീണാ ജോർജ് എന്ന ഒരു ആരോഗ്യ മന്ത്രി വന്നതിനു ശേ...,original
3253,ഇതാണ് പിണുവാതിര😂😂😂,original
3254,കേസ് എടുക്കണം 💯👍,original
3255,എല്ലാത്തിനേം 501 സോപ്പിട്ട് കളിപ്പിച്ച് വിടണം,original


In [ ]:
data['label'].value_counts()

,count
label,
original,1658
Fake,1599


In [ ]:
!pip install transformers torch scikit-learn pandas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [ ]:
# Check for empty texts
print(f"Before removing empty texts: {len(data)} rows")

# Filter out rows where the 'Text' column is empty or contains only whitespace
data = data[data['text'].str.strip().astype(bool)]
train2 = train2[train2['text'].str.strip().astype(bool)]

# Check the number of rows after removal
print(f"After removing empty texts: {len(data)} rows")

In [ ]:
import re

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )

    return emoji_pattern.sub(r"", text)

In [ ]:
# @title Default title text
def preprocess_text(text):
    # Remove emojis
    text = remove_emojis(text)
    # Lowercase the text
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove special characters and punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)
train2['text'] = train2['text'].apply(preprocess_text)

In [ ]:
data['label'] = data['label'].map({'Fake': 0, 'original': 1})
# Encode labels
train2['label'] = train2['label'].map({'Fake': 0, 'original': 1})  # Encode labels

In [ ]:
data['label'].value_counts()

In [ ]:
data.info()

In [ ]:
train2['label'].value_counts()

In [ ]:
train2.info()

In [ ]:
model_name = "bert-base-multilingual-cased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_data(tokenizer, texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
    encodings["labels"] = labels  # Add labels to encoding

    return encodings

In [ ]:
X_train = data['text'].tolist()
X_test = train2['text'].tolist()

In [ ]:
y_train = data['label'].tolist()
y_test = train2['label'].tolist()

In [ ]:
train_encodings = tokenize_data(tokenizer, X_train, y_train)
test_encodings = tokenize_data(tokenizer, X_test, y_test)

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset

# Convert encodings to datasets.Dataset
train_dataset = Dataset.from_dict(train_encodings)
test_dataset = Dataset.from_dict(test_encodings)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3257
})

In [ ]:
test_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 815
})

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
from tabulate import tabulate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    accuracy = accuracy_score(labels, preds)

    metrics = {
        "Precision": precision,
        "Recall": recall,
        "Macro F1": f1,
        "Accuracy": accuracy
    }

    table = tabulate(metrics.items(), headers=["Metric", "Value"], tablefmt="grid")
    print(table)

    return {"precision": precision, "recall": recall, "macro_f1": f1, "accuracy": accuracy}

In [ ]:
# RUN ALL THESE BELOW MODELS SEPERATELY,
# SAVE THE NOTEBOOK FOR EACH MODEL WITH THE RESULT
# ALSO SAVE THE SUBMISSION CSV FILE.

# models = {
#     "IndicBERT": ("ai4bharat/indic-bert", "ai4bharat/indic-bert"),
#     "mBERT": ("bert-base-multilingual-cased", "bert-base-multilingual-cased"),
#     "XLM-RoBERTa": ("xlm-roberta-base", "xlm-roberta-base")  # Ensure this line has a closing brace
# }

In [ ]:
training_args = TrainingArguments(
    output_dir="./mbert_results",
    evaluation_strategy="epoch",   # Evaluate at the end of each epoch
    save_strategy="epoch",         # Save model at the end of each epoch
    learning_rate=5e-4,
    save_total_limit=1,            # Keep only the best model
    load_best_model_at_end=True,   # Load the best model after training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,            # Adjust as needed
    logging_dir="./mbert_logs",
    logging_steps=500,
    metric_for_best_model="eval_loss",  # Use validation loss to determine the best model
    greater_is_better=False             # Lower loss is better
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Add this line
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Precision,Recall,Macro F1,Accuracy
1,No log,0.697183,0.249080,0.500000,0.332514,0.498160
2,No log,0.693259,0.250920,0.500000,0.334150,0.501840
3,0.721800,0.695156,0.249080,0.500000,0.332514,0.498160
4,0.721800,0.709107,0.250920,0.500000,0.334150,0.501840
5,0.717600,0.699189,0.250920,0.500000,0.334150,0.501840
6,0.717600,0.696974,0.250920,0.500000,0.334150,0.501840


+-----------+----------+
| Metric    |    Value |
+===========+==========+
| Precision | 0.24908  |
+-----------+----------+
| Recall    | 0.5      |
+-----------+----------+
| Macro F1  | 0.332514 |
+-----------+----------+
| Accuracy  | 0.49816  |
+-----------+----------+


...

Epoch,Training Loss,Validation Loss,Precision,Recall,Macro F1,Accuracy
1,No log,0.697183,0.249080,0.500000,0.332514,0.498160
2,No log,0.693259,0.250920,0.500000,0.334150,0.501840
3,0.721800,0.695156,0.249080,0.500000,0.332514,0.498160
4,0.721800,0.709107,0.250920,0.500000,0.334150,0.501840
5,0.717600,0.699189,0.250920,0.500000,0.334150,0.501840
6,0.717600,0.696974,0.250920,0.500000,0.334150,0.501840
7,0.717600,0.703442,0.250920,0.500000,0.334150,0.501840
8,0.707700,0.695082,0.250920,0.500000,0.334150,0.501840


+-----------+---------+
| Metric    |   Value |
+===========+=========+
| Precision | 0.25092 |
+-----------+---------+
| Recall    | 0.5     |
+-----------+---------+
| Macro F1  | 0.33415 |
+-----------+---------+
| Accuracy  | 0.50184 |
+-----------+---------+


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


+-----------+---------+
| Metric    |   Value |
+===========+=========+
| Precision | 0.25092 |
+-----------+---------+
| Recall    | 0.5     |
+-----------+---------+
| Macro F1  | 0.33415 |
+-----------+---------+
| Accuracy  | 0.50184 |
+-----------+---------+


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
